# T3 - Running scenarios

While running individual sims can be interesting for simple explorations, at some point it will almost always be necessary to run a large number of simulations simultaneously -- to explore different scenarios, to perform calibration, or simply to get uncertainty bounds on a single projection. This tutorial explains how to do that.

## Running with MultiSims

The 

In [ ]:
import sciris as sc
import covasim as cv

def myfunc(x):
    sc.tic()
    a = 0.0
    for i in range(int(1e7)):
        a += float(x)*1e-8
    sc.toc()
    sim = cv.Sim()
    sim.run()
    return a**2

output = sc.parallelize(myfunc, range(10))
print(output)

In [ ]:
import sciris as sc
import covasim as cv

def myfunc(x):
    sc.tic()
    a = x
    for i in range(int(1)):
        a += float(x)*1e-8
    sc.toc()
    sim = cv.Sim(pop_size=100e3)
    sim.run()
    return #a**2

output = sc.parallelize(myfunc, range(4))
print(output)

Covasim 2.0.0 (2020-11-12) — © 2020 by IDM
Initializing sim with 100000 people for 60 days
Initializing sim with 100000 people for 60 daysInitializing sim with 100000 people for 60 days

Initializing sim with 100000 people for 60 days
  Running 2020-03-01 ( 0/60) (3.15 s)  ———————————————————— 2%
  Running 2020-03-01 ( 0/60) (3.18 s)  ———————————————————— 2%

In [ ]:
import covasim as cv
import sciris as sc

def run_sim(label):
    sim = cv.Sim(label=str(label))
    sim.run()
    return sim

sim = sc.parallelize(run_sim, range(3))

In [2]:
import covasim as cv

s1 = cv.Sim()
s2 = cv.Sim()
msim = cv.MultiSim([s1, s2]*4)
msim.run()

Initializing sim with 20000 people for 60 daysInitializing sim with 20000 people for 60 daysInitializing sim with 20000 people for 60 days


Initializing sim with 20000 people for 60 days
Initializing sim with 20000 people for 60 days
Initializing sim with 20000 people for 60 days
Initializing sim with 20000 people for 60 daysInitializing sim with 20000 people for 60 days

  Running "Sim 0": 2020-03-01 ( 0/60) (0.81 s)  ———————————————————— 2%
  Running "Sim 0": 2020-03-01 ( 0/60) (0.83 s)  ———————————————————— 2%
  Running "Sim 0": 2020-03-01 ( 0/60) (0.82 s)  ———————————————————— 2%
  Running "Sim 0": 2020-03-01 ( 0/60) (0.86 s)  ———————————————————— 2%
  Running "Sim 0": 2020-03-11 (10/60) (0.94 s)  •••————————————————— 18%
  Running "Sim 0": 2020-03-11 (10/60) (0.95 s)  •••————————————————— 18%
  Running "Sim 0": 2020-03-11 (10/60) (0.95 s)  •••————————————————— 18%
  Running "Sim 0": 2020-03-11 (10/60) (0.99 s)  •••————————————————— 18%
  Running "Sim 0": 2020-03-21 (20/60) (1.10 

## Running with Scenarios

TBC